In [ ]:
#!/usr/bin/env python
# coding: utf-8

import functools
import nest_asyncio
import os
import traceback
import pandas as pd
import importlib
import json
from flask import Flask, request, jsonify, render_template
from icecream import ic
from IPython import get_ipython
from IPython.utils.capture import capture_output
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated, Sequence
import operator
from langchain import hub
from gsheet_handler import GSheetHandler

nest_asyncio.apply()
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

# Extract example dialog from Google Sheet
example_dialog = ggl.extract_data_from_google_sheet(
    'https://docs.google.com/spreadsheets/d/1sBAFF62fAjSdPrGYTWgpjmBjLRiFCb8DNAX8ZypI-_U/edit?gid=0#gid=0',
    0
)

from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage, AIMessage

def create_training_prompt(dialogs):
    instruction_message = SystemMessage(content="Ты консультант по инвестиционным программам правительства Москвы.")
    examples_message_content = "Примеры диалогов:\n"
    
    for i, row in dialogs.iterrows():
        examples_message_content += f"Человек: {row['Human']}\nAI: {row['AI']}\n"

    examples_message = SystemMessage(content=examples_message_content)
    
    messages = [instruction_message, examples_message]
    prompt_template = ChatPromptTemplate.from_messages(messages)
    return prompt_template

# Create training prompt
training_prompt = create_training_prompt(example_dialog)
formatted_messages = training_prompt.format_messages()

for message in formatted_messages:
    print(f"{message.type}: {message.content}")

# Load config
with open('config.json') as config_file:
    config = json.load(config_file)

os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "nlmk_bot"

# Initialize ChatOpenAI model
chat = ChatOpenAI()

def create_training_prompt(dialogs):
    instruction_message = SystemMessage(content="Ты консультант по инвестиционным программам правительства Москвы.")
    examples_message_content = "Примеры диалогов:\n"
    
    for i, row in dialogs.iterrows():
        examples_message_content += f"Человек: {row['Human']}\nAI: {row['AI']}\n"

    examples_message = SystemMessage(content=examples_message_content)
    
    messages = [instruction_message, examples_message]
    prompt_template = ChatPromptTemplate.from_messages(messages).format_messages()
    return prompt_template


training_messages = create_training_prompt(example_dialog)

# History of messages
message_history = training_messages.copy()

# Function to create a prompt with history
def create_prompt_with_history(history, user_input):
    messages = history + [HumanMessage(content=user_input)]
    return ChatPromptTemplate.from_messages(messages)

# Flask app
app = Flask(__name__, static_folder='static', template_folder='templates')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat_endpoint():
    user_message = request.json.get('message', '')
    ic(f"Received message: {user_message}")
    try:
        # Create prompt with history and user input
        prompt_with_history = create_prompt_with_history(message_history, user_message)
        formatted_messages = prompt_with_history.format_messages(user_input=user_message)
        response = chat.invoke(formatted_messages)
        ic(f"Generated response: {response.content}")
        
        # Add the new user message and AI response to the history
        message_history.append(HumanMessage(content=user_message))
        message_history.append(AIMessage(content=response.content))
        
        return jsonify({'response': response.content})
    except Exception as e:
        error_message = f"Произошла ошибка: {e}"
        ic(error_message)
        traceback.print_exc()
        return jsonify({'response': error_message}), 500

if __name__ == "__main__":
    ic("Starting Flask app")
    app.run(host='0.0.0.0', port=5000)


no docx
no docx
system: Ты консультант по инвестиционным программам правительства Москвы.
system: Примеры диалогов:
Человек: Интересует программа «Земля за 1 рубль для производства»
AI: Это программа для создания новых производств на территории города Москвы. По программе «Земля за 1 рубль для производства» арендная плата на весь срок аренды земельного участка, оформляемого для проектирования и строительства/реконструкции объектов промышленно-производственного назначения, устанавливается в размере 1 (одного) рубля в год. 
В случае возникновения дополнительных вопросов готовы переключить Вас на соответствующего специалиста.
Человек: Где найти более подробную информацию о конкретном земельном участке?
AI: Ознакомиться с земельными участками, потенциально доступными для строительства производственных объектов, можно на Инвестиционном портале Москвы. Для этого необходимо на главной странице Инвестиционного портала города Москвы investmoscow.ru (инвестмоскоу точка ру) выбрать пункт 4 «Земля

ic| 'Starting Flask app'


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://45.133.178.134:5000
Press CTRL+C to quit
185.155.16.157 - - [16/Jun/2024 10:10:27] "GET / HTTP/1.1" 200 -
185.155.16.157 - - [16/Jun/2024 10:10:27] "GET /static/AI_Robot_02_3d.svg HTTP/1.1" 200 -
ic| f"Received message: {user_message}": 'Received message: Подобрать решение для бизнеса'
185.155.16.157 - - [16/Jun/2024 10:10:32] "GET /favicon.ico HTTP/1.1" 404 -
ic| f"Generated response: {response.content}": ('Generated response: Для подбора оптимального решения для вашего бизнеса, мне '
                                                'нужно больше информации о вашем предприятии, вашей отрасли, целях и задачах. '
                                                'Могли бы вы поделиться более подробной информацией о вашем бизнесе и том, '
                                                'что именно вас интересует?')
185.155.16.157 - - [16/Jun/2024 10:10:33] "POST /chat HTTP/1.1" 200 -
ic| f"Receive

In [ ]:
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

#меры поддержки для бизнеса от правительства Москвы
support = ggl.extract_data_from_google_sheet(
    'https://docs.google.com/spreadsheets/d/1C3m50L3iUWhkkrp5CWEmksVRAkA8y4hX6m2tYgS8-4I/edit?gid=1891107571#gid=1891107571',
    0)
support